# BRW Estimation

Replication code of the BRW shock by Bu, Wu, and Rogers (2021).

In [259]:
import pandas as pd
import statsmodels.api as sm
from linearmodels.iv import IV2SLS


maturity = 2
year = 2020
month = 12
monthname = "Dec"
MAR2020 = 0

## 1. Calculating the difference of the 2-year Treasury series (DGS2)

Load the data and rename columns

In [260]:
data = pd.read_excel(r"C:\Users\Alysson\Documents\GitHub\Monetary-Shocks\USA\DGS2.xls")
data.rename(columns={"observation_date": "date"}, inplace=True)
data.sort_values(by='date', inplace=True)
data.replace(0, method='ffill', inplace=True)
data.head()

,date,DGS2
0,1994-01-03,4.30
1,1994-01-04,4.29
2,1994-01-05,4.31
3,1994-01-06,4.24
4,1994-01-07,4.09


Convert date to datetime format and create additional date-related columns

In [261]:
data["date"] = pd.to_datetime(data["date"])
data["year"] = data["date"].dt.year
data["month"] = data["date"].dt.month
data["day"] = data["date"].dt.day
data["mdate"] = data["date"].dt.to_period("M")

Filter data for the specified period (up to September 2019 - Cut-off date used for the estimation in the research paper.)

In [262]:
data = data[data["mdate"] <= pd.Period("2019-09")]
data = data[data["mdate"] >= pd.Period("1994-01")]

Calculate the difference of the series (dgs_d)

In [263]:
data["dgs_d"] = data["DGS" + str(maturity)] - data["DGS" + str(maturity)].shift(1)
data.head()

,date,DGS2,year,month,day,mdate,dgs_d
0,1994-01-03,4.30,1994,1,3,1994-01,NaN
1,1994-01-04,4.29,1994,1,4,1994-01,-0.01
2,1994-01-05,4.31,1994,1,5,1994-01,0.02
3,1994-01-06,4.24,1994,1,6,1994-01,-0.07
4,1994-01-07,4.09,1994,1,7,1994-01,-0.15


Set "mdate" as the index and convert it to a time series

In [264]:
data.set_index("mdate", inplace=True)
data.index = pd.to_datetime(data.index.to_timestamp())

Drop unnecessary columns

In [265]:
data.drop(columns=["year", "month", "day"], inplace=True)
data.head()

,date,DGS2,dgs_d
mdate,,,
1994-01-01,1994-01-03,4.30,NaN
1994-01-01,1994-01-04,4.29,-0.01
1994-01-01,1994-01-05,4.31,0.02
1994-01-01,1994-01-06,4.24,-0.07
1994-01-01,1994-01-07,4.09,-0.15


Save the resulting DataFrame to a new file

In [266]:
data.to_pickle("temp.pkl")

## 2. Adding FOMC dates

Load the FOMC dates from the Excel file

In [267]:
fomc_dates = pd.read_excel("C:/Users/Alysson/Documents/GitHub/Monetary-Shocks/USA/FOMCdate.xlsx")
fomc_dates.rename(columns={"date": "FOMC_date"}, inplace=True)
fomc_dates.head()

,FOMC_date,q
0,1936-03-19,1
1,1936-05-25,1
2,1936-11-20,1
3,1937-01-26,1
4,1937-03-15,1


Convert date to datetime format and create additional date-related columns

In [268]:
fomc_dates["FOMC_date"] = pd.to_datetime(fomc_dates["FOMC_date"])
fomc_dates["year"] = fomc_dates["FOMC_date"].dt.year
fomc_dates["month"] = fomc_dates["FOMC_date"].dt.month
fomc_dates["day"] = fomc_dates["FOMC_date"].dt.day
fomc_dates["mdate"] = fomc_dates["FOMC_date"].dt.to_period("M")

MAR2020 = 0

if MAR2020 == 0:
    fomc_dates = fomc_dates[~((fomc_dates["year"] == 2020) & (fomc_dates["month"] == 3) & (fomc_dates["day"] == 15))]


Drop unnecessary columns

In [269]:
fomc_dates.drop(columns=["year", "month", "day"], inplace=True)

Load the previous DataFrame "temp.dta" 

In [270]:
temp_data = pd.read_pickle("temp.pkl")

Merge the two DataFrames on the 'date' column

In [271]:
merged_data = pd.merge(temp_data, fomc_dates, how="inner", left_on="date", right_on="FOMC_date")

Drop rows where 'q' is missing (denoted as NaN)

In [272]:
merged_data = merged_data.dropna(subset=["q"])

Replace missing values (NaN) in 'q' with 0

In [273]:
merged_data["q"].fillna(0, inplace=True)
merged_data.head()

,date,DGS2,dgs_d,FOMC_date,q,mdate
0,1994-02-04,4.42,0.14,1994-02-04,1,1994-02
1,1994-02-11,4.44,-0.01,1994-02-11,0,1994-02
2,1994-03-22,5.03,-0.06,1994-03-22,1,1994-03
3,1994-03-29,5.19,0.07,1994-03-29,0,1994-03
4,1994-05-17,5.88,-0.15,1994-05-17,1,1994-05


Save the merged DataFrame to a new file

In [274]:
merged_data.to_pickle("temp.pkl")

## 3. Estimation

Load the data from "feds200628.csv" 

In [275]:
yield_data = pd.read_csv(r"C:\Users\Alysson\Documents\GitHub\Monetary-Shocks\USA\feds200628.csv")

Convert 'date' column to datetime format and keeping only necessary columns

In [276]:
yield_data['date'] = pd.to_datetime(yield_data['Date'], format='%Y-%m-%d')
yield_data.sort_values(by='date', inplace=True)
yield_data.loc[:, 'SVENY01':'SVENY30'] = yield_data.loc[:, 'SVENY01':'SVENY30'].fillna(method='ffill')

In [277]:
yield_data['year'] = yield_data['date'].dt.year
yield_data['month'] = yield_data['date'].dt.month
yield_data['day'] = yield_data['date'].dt.day
yield_data['date'] = pd.to_datetime(yield_data[['year', 'month', 'day']])
yield_data['mdate'] = yield_data['date'].dt.to_period('M')
yield_data = yield_data[['date', 'mdate'] + [col for col in yield_data.columns if col.startswith('SVENY')]]
yield_data = yield_data[yield_data['mdate'] >= pd.Period('1994-01')]
yield_data.head()

,date,mdate,SVENY01,SVENY02,SVENY03,SVENY04,SVENY05,SVENY06,SVENY07,SVENY08,...,SVENY21,SVENY22,SVENY23,SVENY24,SVENY25,SVENY26,SVENY27,SVENY28,SVENY29,SVENY30
8493,1994-01-03,1994-01,3.731411,4.260040,4.682096,5.024778,5.306916,5.541907,5.739533,5.907118,...,6.848436,6.878646,6.906286,6.931662,6.955033,6.976625,6.996629,7.015213,7.032521,7.048678
8494,1994-01-04,1994-01,3.713510,4.239907,4.656361,4.992733,5.269173,5.499619,5.693969,5.859453,...,6.810627,6.841833,6.870407,6.896656,6.920843,6.943197,6.963913,6.983162,7.001092,7.017833
8495,1994-01-05,1994-01,3.705392,4.248564,4.671605,5.010070,5.286970,5.517565,5.712289,5.878515,...,6.846719,6.878791,6.908162,6.935146,6.960013,6.982997,7.004297,7.024090,7.042527,7.059742
8496,1994-01-06,1994-01,3.668817,4.193361,4.607845,4.943227,5.219876,5.451570,5.647936,5.815925,...,6.794964,6.827326,6.856961,6.884186,6.909274,6.932459,6.953947,6.973913,6.992512,7.009876
8497,1994-01-07,1994-01,3.571147,4.060404,4.454530,4.779420,5.051982,5.283647,5.482460,5.654320,...,6.682563,6.717051,6.748641,6.777669,6.804423,6.829152,6.852073,6.873372,6.893213,6.911738


Create a list for all maturities of the treasuries

Calculate differences for each maturity

In [278]:
list_maturities = [str(i).zfill(2) for i in range(1, 31)]

for num in list_maturities:
    col_name = f'SVENY{num}'
    yield_data[f'{col_name}_d'] = yield_data[col_name].diff()
selected_columns = yield_data.iloc[:, -30:]

Create aligned_dgs_d column

In [279]:
data_complete = merged_data.merge(yield_data, on='date', how='inner')

data_complete.head()

,date,DGS2,dgs_d,FOMC_date,q,mdate_x,mdate_y,SVENY01,SVENY02,SVENY03,...,SVENY21_d,SVENY22_d,SVENY23_d,SVENY24_d,SVENY25_d,SVENY26_d,SVENY27_d,SVENY28_d,SVENY29_d,SVENY30_d
0,1994-02-04,4.42,0.14,1994-02-04,1,1994-02,1994-02,3.865749,4.369629,4.744739,...,0.035375,0.028441,0.022022,0.016079,0.010570,0.005457,0.000704,-0.003722,-0.007851,-0.011711
1,1994-02-11,4.44,-0.01,1994-02-11,0,1994-02,1994-02,3.900780,4.405665,4.795111,...,-0.032590,-0.032605,-0.032620,-0.032637,-0.032654,-0.032672,-0.032691,-0.032709,-0.032727,-0.032745
2,1994-03-22,5.03,-0.06,1994-03-22,1,1994-03,1994-03,4.443199,5.001075,5.396750,...,-0.076686,-0.075292,-0.074018,-0.072847,-0.071769,-0.070774,-0.069852,-0.068995,-0.068197,-0.067453
3,1994-03-29,5.19,0.07,1994-03-29,0,1994-03,1994-03,4.526711,5.186564,5.626675,...,0.082017,0.081543,0.081109,0.080710,0.080344,0.080005,0.079691,0.079400,0.079129,0.078875
4,1994-05-17,5.88,-0.15,1994-05-17,1,1994-05,1994-05,5.301464,5.864555,6.191343,...,-0.172195,-0.166615,-0.161111,-0.155718,-0.150464,-0.145369,-0.140451,-0.135722,-0.131189,-0.126857


### 2SLS

In [216]:
maxt = len(data_complete["date"])
shift = 30
aligned_dgs_d_list = []

In [314]:

# Criar DataFrame com as variáveis
data = pd.DataFrame({'dgs_d_q0': dgs_d_q0, 'dgs_d_q1': dgs_d_q1, 'sveny05_q1': sveny05_q1})

# Definir variáveis endógenas, exógenas e instrumentos
exog = data[['dgs_d_q0']]
endog = data[['dgs_d_q1']]
instruments = data[['dgs_d_q1']]

# Aplicar 2SLS
results = IV2SLS(dependent=data['sveny05_q1'], endog=endog, exog=exog, instruments=instruments).fit()

# Imprimir resultados
print(results)


                          IV-2SLS Estimation Summary                          
Dep. Variable:             sveny05_q1   R-squared:                      0.7108
Estimator:                    IV-2SLS   Adj. R-squared:                 0.7080
No. Observations:                 211   F-statistic:                    178.83
Date:                Fri, Aug 25 2023   P-value (F-stat)                0.0000
Time:                        13:11:43   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
dgs_d_q0      -0.0257     0.0757    -0.3390     0.7346     -0.1741      0.1227
dgs_d_q1       0.9777     0.0739     13.233     0.00